# Final Presentation ACDC

In this notebook we will walk through examples of this package. The package consists of a class for graph processing, a class for processing the power system and a class to simulate the power system, which were made in the three assignments. The first two parts are individual modules. The third part uses the first and second module. They cover the following points:

Graph processing:
- Find downstream vertices.
- Find alternative edges.

Power grid model:
- Handle PGM input format.
- Aggregate the power flow results in tables. 
- Show tables with data of powerflow calculations.

Power system simulation:
- Calculate EV penetration levels. 
- Calculate optimal tap positions. 
- Do N-1 calculations. 

All:
- Raise errors if the data input is incorrect.

The package makes use of the Power Grid Model library.

In [33]:
# some basic imports
import pytest
import graph_processing as tp  # Import power_system_simpulation.graphy_processing
import pandas as pd
import numpy as np
import scipy as sp

import power_flow_processing as pfp
import power_system_simulation as pss
from graph_processing import (
    EdgePairNotUniqueError,
    GraphCycleError,
    GraphNotFullyConnectedError,
    GraphProcessor,
    IDNotFoundError,
    IDNotUniqueError,
    InputLengthDoesNotMatchError,
)


# 1.1 Input dataset

We create an input graph by using the following parameters: 
- `vertex_ids`
- `edge_ids`
- `edge_vertex_id_pairs`
- `edge_enabled`
- `source_vertex_id`

In [34]:
vertex_ids = [0, 2, 4, 6, 10]  # All unique vertex ids
edge_ids = [1, 3, 5, 7, 8, 9]  # All unique edge ids
edge_vertex_id_pairs = [ # Which vertex ids are connected by an edge
        (0, 2),  # edge 1
        (0, 4),  # edge 3
        (0, 6),  # edge 5
        (2, 4),  # edge 7
        (4, 6),  # edge 8
        (2, 10),  # edge 9
    ]
edge_enabled = [True, True, True, False, False, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

This graph will result in this visual representation: 

In [35]:
'''      
vertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10
                 |                               |
                 |                           edge_7(disabled)
                 |                               |
                 -----------edge_3(enabled)-- vertex_4
                 |                               |
                 |                           edge_8(disabled)
                 |                               |
                 -----------edge_5(enabled)-- vertex_6
'''

'      \nvertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10\n                 |                               |\n                 |                           edge_7(disabled)\n                 |                               |\n                 -----------edge_3(enabled)-- vertex_4\n                 |                               |\n                 |                           edge_8(disabled)\n                 |                               |\n                 -----------edge_5(enabled)-- vertex_6\n'

# 1.2 Validation
The error handling is done by raising exceptions. The graph is tested for the following conditions: 
1. `vertex_ids` and `edge_ids` should be unique.
    - This function compares the length of all `vertex_ids` to the set of `vertex_ids` and gives an error if they are not the same. It uses the same approach for `edge_ids`.
2. `edge_vertex_id_pairs` should have the same length as `edge_ids`.
    - This function compares the length of the list of `edge_vertex_id_pairs` and `edge_ids`.
3. `edge_vertex_id_pairs` should contain valid `vertex ids`.
    - Using a loop all `edge_vertex_id_pairs` are checked to also be valid `edge_ids`.
4. `edge_enabled` should have the same length as `edge_ids`.
    - The length of the `edge_enabled` list is compared to length of the `edge_ids` list. 
5. `source_vertex_id` should be a valid `vertex_ids`.
     - The `source_vertex_id` is checked to be part of `vertex_ids`.
6. The graph should not contain cycles.
    - An adjacency list is built and using depth first search cycles are detected. 
7. The graph should be fully connected.
    - Using depth first search the length of all the visited vertex and the length of `vertex_ids` is compared. 
8. Multiple edges should not connect the same two `vertex_ids`. 
    - A list of `edge_vertex_id_pairs` is compared to a list of the set of `edge_vertex_id_pairs`.

This can be tested by calling the `tp.GraphProcessor` function: 

In [36]:
test2 = tp.GraphProcessor(
    vertex_ids=vertex_ids,
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id_pairs,
    edge_enabled=edge_enabled,
    source_vertex_id=source_vertex_id,
    )

# Example
When for example not all `vertex_ids` are unique the following error will be raised: 

In [37]:
try:
    GraphProcessor([1, 2, 3, 3, 5], [1, 2, 3], [(1, 2), (2, 3), (1, 5)], [True, True, True], 1)
except IDNotUniqueError as e:
    print(e)

Vertex IDs are not unique


In [38]:
test = tp.GraphProcessor(
        vertex_ids=vertex_ids,
        edge_ids=edge_ids,
        edge_vertex_id_pairs=edge_vertex_id_pairs,
        edge_enabled=edge_enabled,
        source_vertex_id=source_vertex_id,
    )

# 1.3 Find downstream vertices

 Given an `edge_id`, return all the vertices which are in the downstream of the edge, with respect to the source vertex. Including the downstream vertex of the edge itself!
 Only the `edge_enabled` are taken into account. 
 The function returns a list of all downstream `vertex_ids`. 

In [8]:
vertex_ids = [0, 2, 4]  # All unique vertex ids
edge_ids = [1, 3]  # All unique edge ids
edge_vertex_id_pairs = [(0, 2), (2, 4)]  # Egde 1 and egde 3
edge_enabled = [True, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

test3 = tp.GraphProcessor(
        vertex_ids=vertex_ids,
        edge_ids=edge_ids,
        edge_vertex_id_pairs=edge_vertex_id_pairs,
        edge_enabled=edge_enabled,
        source_vertex_id=source_vertex_id,
    )

downstream_vertices = test3.find_downstream_vertices(1)
print(downstream_vertices)

[2, 4]


# 1.4 Find alternative edges 
Given an enabled edge the following analysis is done: 
- If this edge is going to be disabled. 
- Which (currently disabled) edge can be enabled to ensure that the graph is again fully connected and acyclic?
- Return a list of all alternative edges.

Our example graph would return the following results:   
        Call find_alternative_edges with disabled_edge_id=1 will return [7]   
        Call find_alternative_edges with disabled_edge_id=3 will return [7, 8]   
        Call find_alternative_edges with disabled_edge_id=5 will return [8]   
        Call find_alternative_edges with disabled_edge_id=9 will return []   

This function can be used by using the `find_alternative_edges` function and giving an `edge_ids` as input. 

In [9]:
alternative_edges = test2.find_alternative_edges(1)
print("Alternative edge when disabling edge 1 is:", alternative_edges)

alternative_edges = test2.find_alternative_edges(3)
print("Alternative edge when disabling edge 3 are:", alternative_edges)

alternative_edges = test2.find_alternative_edges(5)
print("Alternative edge when disabling edge 5 are:", alternative_edges)

alternative_edges = test2.find_alternative_edges(9)
print("Alternative edge when disabling edge 9 are:", alternative_edges)

Alternative edge when disabling edge 1 is: [7]
Alternative edge when disabling edge 3 are: [7, 8]
Alternative edge when disabling edge 5 are: [8]
Alternative edge when disabling edge 9 are: []


# 2.1 Power Grid Model (PGM) Input

The following sections describe a power grid calculation module with the use of the `power-grid-model` library. Firstly, we must handle the following inputs:
- A power grid in PGM format
- A table containing active load profile of all the `sym_load` in the grid, with timestamps and load ids.
- A table containing ractive load profile of all the `sym_load` in the grid, with timestamps and load ids.

This data can be imported using the following code:

In [10]:
from power_grid_model.utils import json_deserialize_from_file

grid_data = json_deserialize_from_file("input_network_data.json")
active_power_profile = pd.read_parquet("active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("reactive_power_profile.parquet")

The `grid_data` consists of several different tables containing information about different elements of the grid, like lines, nodes, sources and load information.

In [11]:
print(pd.DataFrame(grid_data['line']))
print(pd.DataFrame(grid_data['node']))
print(pd.DataFrame(grid_data['source']))
print(pd.DataFrame(grid_data['sym_load']))

   id  from_node  to_node  from_status  to_status    r1   x1       c1  tan1  \
0   5          1        2            1          1  0.25  0.2  0.00001   0.0   
1   6          2        3            1          1  0.25  0.2  0.00001   0.0   
2   7          3        4            1          1  0.25  0.2  0.00001   0.0   

   r0  x0  c0  tan0     i_n  
0 NaN NaN NaN   NaN  1000.0  
1 NaN NaN NaN   NaN  1000.0  
2 NaN NaN NaN   NaN  1000.0  
   id  u_rated
0   1  10500.0
1   2  10500.0
2   3  10500.0
3   4  10500.0
   id  node  status  u_ref  u_ref_angle           sk  rx_ratio  z01_ratio
0  11     1       1    1.0          NaN  200000000.0       NaN        NaN
   id  node  status  type  p_specified  q_specified
0   8     2       1     0          0.0          0.0
1   9     3       1     0          0.0          0.0
2  10     4       1     0          0.0          0.0


The `active_power_profile` and `reactive_power_profile` contain the power profiles for different loads, so they tell how much power should be supplied to for example a household or factory (loads) at what time.

In [12]:
print(active_power_profile)
print(reactive_power_profile)

Load_ID                         8              9              10
Timestamp                                                       
2024-01-01 00:00:00   97627.007855  430378.732745  205526.752143
2024-01-01 01:00:00   89766.365994 -152690.401322  291788.226133
2024-01-01 02:00:00 -124825.577475  783546.001564  927325.521002
2024-01-01 03:00:00 -233116.962348  583450.076165   57789.839506
2024-01-01 04:00:00  136089.122188  851193.276585 -857927.883604
2024-01-01 05:00:00 -825741.400597 -959563.205119  665239.691096
2024-01-01 06:00:00  556313.501900  740024.296494  957236.684466
2024-01-01 07:00:00  598317.128433  -77041.275494  561058.352573
2024-01-01 08:00:00 -763451.148262  279842.042655 -713293.425182
2024-01-01 09:00:00  889337.834099   43696.643500 -170676.120019
Load_ID                         8              9              10
Timestamp                                                       
2024-01-01 00:00:00 -470888.775791  548467.378868  -87699.335567
2024-01-01 01:00:00  1368

# 2.2 Constructing PGM using input data

Using the previously imported input data, a power grid model (PGM) can be constructed. Furthermore, validation is important in this step, and as such an error should be raised if the data is invalid. The PGM can be constructed by calling the class `pfp.PowerFlow`. 

In [13]:
pgm = pfp.PowerFlow(grid_data)

The `PowerFlow` class contains an initialization function where the PGM is constructed, and the data is validated using the following line of code, which is already present in the actual function. 

In [14]:
pfp.assert_valid_input_data(input_data=grid_data, symmetric=True, calculation_type=pfp.CalculationType.power_flow)

# 2.3 PGM Batch update dataset and power flow calculation

The batch update dataset includes multiple power profiles (active and reactive power) for various nodes in the grid over a specified period. Instead of updating the grid model for each individual timestamp or node separately, the entire set of updates is applied at once, allowing us to perform the power flow calculation for the entire period.

Within the `batch_powerflow` function, the power flow calculation is also performed. This is done using the Newton-Raphson method, which returns the `output_data` which contains the solution to the power flow calculation. Furthermore, the batch update dataset is also validated within the function.

This entire functionality can easily be performed with the following line of code:

In [15]:
output_data = pgm.batch_powerflow(active_power_profile, reactive_power_profile)

# 2.4 Aggregating Power Flow Results

Now all the data has been prepared and the state of the power grid after applying all batch updates has been computed, the power flow results will be aggregated in two tables:
- A voltage table with each row representing a timestamp and containing the following columns:
    - Timestamp (index column)
    - Maximum p.u. voltage of all the nodes for this timestamp
    - The node ID with the maximum p.u. voltage
    - Minimum p.u. voltage of all the nodes for this timestamp
    - The node ID with the minimum p.u. voltage

- A loading table with each row representing a line and containing the following columns:
    - Line ID (index column)
    - Total energy loss of the line in kWh over the entire period
    - Maximum loading in p.u. of the line across the whole timeline
    - Timestamp of this maximum loading moment
    - Minimum loading in p.u. of the line across the whole timeline
    - Timestamp of this minimum loading moment

These tables are constructed using two separate functions, which also gives the results for the given input data and power profiles:

In [16]:
pgm.aggregate_voltage_table(active_power_profile, reactive_power_profile)

,Max_Voltage,Max_Voltage_Node,Min_Voltage,Min_Voltage_Node
Timestamp,,,,
2024-01-01 00:00:00,1.004847,1,1.003450,3
2024-01-01 01:00:00,1.012053,3,1.007998,1
2024-01-01 02:00:00,0.997474,1,0.984365,4
2024-01-01 03:00:00,1.006557,4,1.005190,1
2024-01-01 04:00:00,1.011007,4,1.005877,1
2024-01-01 05:00:00,1.020486,4,1.010570,1
2024-01-01 06:00:00,1.006342,1,0.998868,4
2024-01-01 07:00:00,1.004306,1,1.002822,3
2024-01-01 08:00:00,1.020402,4,1.010501,1


In [17]:
pgm.aggregate_loading_table(active_power_profile, reactive_power_profile)

,Total_Loss,Max_Loading,Max_Loading_Timestamp,Min_Loading,Min_Loading_Timestamp
Line_ID,,,,,
5,63.294763,0.149830,2024-01-01 06:00:00,0.063798,2024-01-01 03:00:00
6,36.775143,0.111039,2024-01-01 05:00:00,0.037184,2024-01-01 00:00:00
7,14.872359,0.071700,2024-01-01 02:00:00,0.020380,2024-01-01 01:00:00


# 3.1 Using a LV grid with a MV/LV transformer
This part of the package will present low voltage grid analytics functions. These analytics include EV penetration level and optimal tap position. Input data can be given as: 
- A LV grid in PGM input format.
- LV feeder IDs list.
- A (active and reactive) load profile.
- A pool of EV charging profiles for the same time period as the time period of load profile.

# 3.2 Validation
The data is validated to check for the following criteria: 
1. The LV grid should be a valid PGM input data.
    - Is checked when initializing the data with the power grid calculation module.
2. The LV grid has exactly one `transformer`, and one `source`.
    - The length of `transformer`and `source` is checked to be equal to 1.
3. All IDs in the LV Feeder IDs are valid line IDs.
    - Using a loop the IDs of the LV Feeder are checked to be IDs of `line`.
4. All the lines in the LV Feeder IDs have the `from_node` the same as the `to_node` of the `transformer`.
    - With a loop, the `from_node` of the LV Feeder IDs are checked to be the same as the `to_node`of the `transformer`.
5. The grid is fully connected in the initial state.
    - After reformatting the data, this is checked when initializing the data with the graph processing module.
6. The grid has no cycles in the initial state.
    - After reformatting the data, this is checked when initializing the data with the graph processing module.
7. The timestamps are matching between the active load profile and reactive load profile.
    - Is checked when initializing the data with the power grid calculation module.
8. The IDs in active load profile and reactive load profile are matching.
    - Is checked when initializing the data with the power grid calculation module.
9. The IDs in active load profile and reactive load profile are valid IDs of `sym_load`.
    - Is checked when initializing the data with the power grid calculation module.

# Example

A dataset is created where not all LV Feeder IDs are valid line IDs.

In [29]:
from power_grid_model import initialize_array, LoadGenType

# node
node = initialize_array("input", "node", 3)
node["id"] = [2, 4, 6]
node["u_rated"] = [1e4, 4e2, 4e2]

# load
sym_load = initialize_array("input", "sym_load", 1)
sym_load["id"] = [7]
sym_load["node"] = [6]
sym_load["status"] = [1]
sym_load["type"] = [LoadGenType.const_power]
sym_load["p_specified"] = [1e3]
sym_load["q_specified"] = [5e3]

# source
source = initialize_array("input", "source", 1)
source["id"] = [1]
source["node"] = [2]
source["status"] = [1]
source["u_ref"] = [1.0]

# line
line = initialize_array("input", "line", 1)
line["id"] = [5]
line["from_node"] = [4]
line["to_node"] = [6]
line["from_status"] = [1]
line["to_status"] = [1]
line["r1"] = [10.0]
line["x1"] = [0.0]
line["c1"] = [0.0]
line["tan1"] = [0.0]

# transformer
transformer = initialize_array("input", "transformer", 1)
transformer["id"] = [3]
transformer["from_node"] = [2]
transformer["to_node"] = [4]
transformer["from_status"] = [1]
transformer["to_status"] = [1]
transformer["u1"] = [1e4]
transformer["u2"] = [4e2]
transformer["sn"] = [1e5]
transformer["uk"] = [0.1]
transformer["pk"] = [1e3]
transformer["i0"] = [1.0e-6]
transformer["p0"] = [0.1]
transformer["winding_from"] = [2]
transformer["winding_to"] = [1]
transformer["clock"] = [5]
transformer["tap_side"] = [0]
transformer["tap_pos"] = [3]
transformer["tap_min"] = [-11]
transformer["tap_max"] = [9]
transformer["tap_size"] = [100]

# all
input_data = {"node": node, "line": line, "sym_load": sym_load, "source": source, "transformer": transformer}

# lv feeders
lv_feeders = [2]

In this case the InvalidLVFeederIDError is raised.

In [32]:
from power_system_simulation import PowerSim, InvalidLVFeederIDError
try:
    PowerSim(input_data, lv_feeders)
except InvalidLVFeederIDError as e:
    print(e)

LV feeder IDs are not valid line IDs


# 3.3 Tap position
In this functionality, the tap position is optimize of the transformer in the LV grid
- The functionality returns the optimal tap position of the transformer by repeating time-series power flow calculation of the whole time periode for all possible tap positions.
- After the power flow calculation the optimal tap position will return by:
    - The minimal total energy loss of all the lines and the whole time period.
    - The minimal deviation of p.u. node voltages with respect to 1 p.u.



